In [2]:
import numpy as np
import pandas as pd

In [3]:
skill_set = ['math','science','music','guitar','piano','singing','presentation','chemistry','physics']

In [4]:
dict = [{'ID':1,'country':'Sri Lanka','skill_set':['math','science','music','guitar']},
        {'ID':2,'country':'England','skill_set':['singing','presentation']},
        {'ID':3,'country':'USA','skill_set':['music','guitar','piano']},
        {'ID':4,'country':'Sri Lanka','skill_set':['presentation','chemistry','physics']},
        {'ID':5,'country':'Japan','skill_set':['math','music','guitar','piano']}
]

In [5]:
df = pd.DataFrame(dict)

In [6]:
df

,ID,country,skill_set
0,1,Sri Lanka,"[math, science, music, guitar]"
1,2,England,"[singing, presentation]"
2,3,USA,"[music, guitar, piano]"
3,4,Sri Lanka,"[presentation, chemistry, physics]"
4,5,Japan,"[math, music, guitar, piano]"


In [7]:
df_expanded_v1 = df["skill_set"].apply(pd.Series)


In [8]:
df_expanded_v1

,0,1,2,3
0,math,science,music,guitar
1,singing,presentation,NaN,NaN
2,music,guitar,piano,NaN
3,presentation,chemistry,physics,NaN
4,math,music,guitar,piano


In [9]:
#importing MultiLabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
#instantiating MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [10]:
df_v2 = pd.DataFrame(mlb.fit_transform(df['skill_set']),columns=mlb.classes_)

In [11]:
df_v2

,chemistry,guitar,math,music,physics,piano,presentation,science,singing
0,0,1,1,1,0,0,0,1,0
1,0,0,0,0,0,0,1,0,1
2,0,1,0,1,0,1,0,0,0
3,1,0,0,0,1,0,1,0,0
4,0,1,1,1,0,1,0,0,0


In [13]:
df_merge_col = pd.merge(df, df_v2, left_index=True, right_index=True)

In [14]:
df_merge_col

,ID,country,skill_set,chemistry,guitar,math,music,physics,piano,presentation,science,singing
0,1,Sri Lanka,"[math, science, music, guitar]",0,1,1,1,0,0,0,1,0
1,2,England,"[singing, presentation]",0,0,0,0,0,0,1,0,1
2,3,USA,"[music, guitar, piano]",0,1,0,1,0,1,0,0,0
3,4,Sri Lanka,"[presentation, chemistry, physics]",1,0,0,0,1,0,1,0,0
4,5,Japan,"[math, music, guitar, piano]",0,1,1,1,0,1,0,0,0


In [16]:
dataset = df_merge_col.drop(['skill_set'],axis=1)

In [17]:
dataset

,ID,country,chemistry,guitar,math,music,physics,piano,presentation,science,singing
0,1,Sri Lanka,0,1,1,1,0,0,0,1,0
1,2,England,0,0,0,0,0,0,1,0,1
2,3,USA,0,1,0,1,0,1,0,0,0
3,4,Sri Lanka,1,0,0,0,1,0,1,0,0
4,5,Japan,0,1,1,1,0,1,0,0,0


In [67]:
def preprocess(dict):
  df = pd.DataFrame(dict)

  mlb = MultiLabelBinarizer()
  df_v2 = pd.DataFrame(mlb.fit_transform(df['skill_set']),columns=mlb.classes_)
  df_merge_col = pd.merge(df, df_v2, left_index=True, right_index=True)
  dataset = df_merge_col.drop(['skill_set'],axis=1)

  return dataset

In [89]:
def find_tutors(student,tutors_df,country=False,n=5):
  '''
  input: pandas series of a specific student entry
  country: whether to consider the country or not
  returns: similar tutors
  '''
  if country:
    tutors_df = tutors_df[tutors_df['country']==student['country']]

  student = student.drop(['country','ID'])
  tutors_df = tutors_df.drop(['country','ID'],axis=1)

  mask = sum([tutors_df==student])
  print(mask)
  sim_scores = mask.sum(axis=1)

  return sim_scores.nlargest(n)


In [46]:
st1 = dataset.iloc[2].drop(['country','ID'])
st2 = dataset.iloc[3].drop(['country','ID'])
dataset_v2 = dataset.drop(['country','ID'],axis=1)

In [47]:
st_mask = [dataset_v2==st2]

In [48]:
st_mask

[   chemistry  guitar   math  music  ...  piano  presentation  science  singing
 0      False   False  False  False  ...   True         False    False     True
 1      False    True   True   True  ...   True          True     True    False
 2      False   False   True  False  ...  False         False     True     True
 3       True    True   True   True  ...   True          True     True     True
 4      False   False  False  False  ...  False         False     True     True
 
 [5 rows x 9 columns]]

In [66]:
st_mask2 = sum(st_mask)
print(st_mask2.sum(axis=1))
scores = st_mask2.sum(axis=1)
print(scores.idxmax())
print(scores.nlargest(3))
# print(sum(sum(st_mask)))
# print(st_mask2)

0    2
1    6
2    3
3    9
4    2
dtype: int64
3
3    9
1    6
2    3
dtype: int64


In [90]:
dict2 = [{'ID':1,'country':'Sri Lanka','skill_set':['math','guitar']},
        {'ID':2,'country':'England','skill_set':['singing','presentation']},
        {'ID':3,'country':'USA','skill_set':['guitar','piano']},
        {'ID':4,'country':'Sri Lanka','skill_set':['presentation','physics']},
        {'ID':5,'country':'USA','skill_set':['math','piano']},
        {'ID':6,'country':'India','skill_set':['math','piano','singing','guitar']}
]

In [91]:
tutors_df = preprocess(dict2)

In [92]:
student = dataset.iloc[2]
tutors_df[tutors_df['country']==student['country']]

,ID,country,guitar,math,physics,piano,presentation,singing
2,3,USA,1,0,0,1,0,0
4,5,USA,0,1,0,1,0,0


In [93]:
similat_tuts = find_tutors(student,tutors_df)

   chemistry  guitar  math  music  ...  piano  presentation  science  singing
0          0       1     0      0  ...      0             1        0        1
1          0       0     1      0  ...      0             0        0        0
2          0       1     1      0  ...      1             1        0        1
3          0       0     1      0  ...      0             0        0        1
4          0       0     0      0  ...      1             1        0        1
5          0       1     0      0  ...      1             1        0        0

[6 rows x 9 columns]


In [86]:
similat_tuts

2    6
0    4
4    4
5    4
1    2
dtype: int64

In [87]:
tutors_df

,ID,country,guitar,math,physics,piano,presentation,singing
0,1,Sri Lanka,1,1,0,0,0,0
1,2,England,0,0,0,0,1,1
2,3,USA,1,0,0,1,0,0
3,4,Sri Lanka,0,0,1,0,1,0
4,5,USA,0,1,0,1,0,0
5,6,India,1,1,0,1,0,1


In [88]:
student

ID                3
country         USA
chemistry         0
guitar            1
math              0
music             1
physics           0
piano             1
presentation      0
science           0
singing           0
Name: 2, dtype: object